In [55]:
import os
import pandas as pd
import pickle

FILTER = "no_rd" # "ln_only", "d_only", "rd_only", "no_dropout"

#exp_name = 'preprocessing_type'
#experiments = ["adam", "other", "rmsprop", "sgd"]
grid_search_results = []
results_path = os.path.join('mac_logs', 'general_norm_drop','gridsearch_results.pkl')
with open(results_path, 'rb') as f:
    partial_results = pickle.load(f)
grid_search_results.extend(partial_results)
print(grid_search_results[0].keys())
    
    

dict_keys(['model-layer_normalization', 'model-rnn_cell_params-recurrent_dropout', 'model-rnn_cell_params-dropout', 'metrics_per_fold'])


In [56]:
import pickle

with open('mac_logs/general_norm_drop/basic_parameters.pickle', 'rb') as f:
    basic_parameters = pickle.load(f)
print(basic_parameters)

{'dataset': {'data_path': '/Users/maderajan/Programming/LSTM-crystal-growth/python/data', 'scaler': StandardScaler(), 'preprocessing_function': <function lag_features at 0x000001B1471860C0>, 'shuffle': True, 'final_testing': False, 'test_size': 0.0, 'outputs_lag': 3, 'inputs_lag': 0, 'window_size': 3, 'return_sequences': True, 'seed': 42, 'scaler_partitions': [[0, 1], [2, 3, 4, 5, 6], [7]], 'scaler_kwargs': {}, 'k_folds': 10}, 'dataset_class': <class 'data_processing.dataset.CrystalDataset'>, 'model': {'bidirectional': False, 'cell_type': 'lstm', 'rnn_cell_params': {'units': 20, 'peephole_connections': False, 'layer_normalization': False, 'ln_epsilon': 0.001, 'ln_center': True, 'ln_scale': True, 'ln_beta_initializer': 'zeros', 'ln_gamma_initializer': 'ones', 'dropout': 0.0, 'recurrent_dropout': 0.0}, 'num_layers': 1, 'rnn_layer_params': {'return_sequences': True}, 'dilation_base': None, 'residual_block_size': None, 'smyl_std': False, 'smyl_residual': False}, 'training': {'batch_size': 

In [57]:
print(len(grid_search_results))
for exp in grid_search_results:
    for key in exp.keys():
        if key != 'metrics_per_fold':
            print(f"{key}: {exp[key]}")
    #print(f"epochs: {exp["training-epochs"]}")
    folds = exp["metrics_per_fold"]
    for i, fold in enumerate(folds):
        print(f"\tFold {i+1}:")
        for metric, values in fold.items():
            scaled_val = values["scaled"]
            unscaled_val = values["unscaled"]
            print(f"\t\t{metric}: {scaled_val.metric:.6f} ({unscaled_val.metric:.6f})\n\t\t\tper_sample s (u):\n\t\t\t\t {scaled_val.metric_per_sample}\n\t\t\t\t({unscaled_val.metric_per_sample})")
    print()
    print("######################################")

50
model-layer_normalization: True
model-rnn_cell_params-recurrent_dropout: 0.0
model-rnn_cell_params-dropout: 0.0
	Fold 1:
		mean_squared_error: 0.000028 (0.997154)
			per_sample s (u):
				 [2.021449e-05, 1.6533264e-05, 2.035702e-05, 1.6556382e-05, 4.181829e-05, 2.0753803e-05, 1.6556683e-05, 2.1518605e-05, 1.9969237e-05, 1.9022624e-05, 3.959353e-05, 4.173934e-05, 2.3670036e-05, 2.8118544e-05, 2.1821015e-05, 2.2009719e-05, 1.9502288e-05, 2.271413e-05, 1.6679145e-05, 1.9467532e-05, 2.4619467e-05, 1.8283365e-05, 1.6612372e-05, 1.9269257e-05, 2.8496039e-05, 6.902958e-05, 4.363789e-05, 2.6551683e-05, 4.65168e-05, 4.167882e-05, 4.0124127e-05, 1.6558613e-05, 2.0858984e-05, 1.8670722e-05, 2.0818547e-05, 5.7201578e-05, 2.0431811e-05, 1.817543e-05, 5.5146316e-05, 1.715376e-05, 6.306758e-05, 1.6812932e-05, 4.0979416e-05, 2.4348717e-05, 1.6594466e-05]
				([0.74740386, 0.68996775, 0.7562215, 0.6879299, 1.1739088, 0.75584567, 0.7019798, 0.77641296, 0.98944587, 0.8334009, 1.0054128, 1.1641011, 1.5

1. 
   - For each experiment, get all samples. 
   - The samples are ordered for each experiment in the same way. 

In [58]:
from sklearn.preprocessing import FunctionTransformer
from types import FunctionType
import re

def get_exp_name(exp):
    name = ""
    for i, key in enumerate(exp.keys()):
        if key != 'metrics_per_fold':
            if i > 0:
                name += " "
            if isinstance(exp[key], str):
                name += exp[key]
            elif isinstance(exp[key], FunctionTransformer):
                return exp[key].func.__name__
            elif isinstance(exp[key], bool):
                name += key if exp[key] else "not "+ key
            elif isinstance(exp[key], FunctionType):
                name += exp[key].__qualname__
            else:
                key_parts = key.split("-")
                last_part = key_parts[-1]
                key_words = last_part.split("_")
                shortcut = "".join([word[0] for word in key_words]) if len(key_words) > 1 else key_words[0]
                name += shortcut + "=" + str(exp[key])

    #name = name.replace("dataset-", "").replace("_", " ").replace("-", " ")
    name = name.capitalize()
    return name

def has_recurrent_dropout(exp):
    for key in exp.keys():
        if "-recurrent_dropout" in key:
            if exp[key] != 0:
                return True
    return False

def has_dropout(exp):
    for key in exp.keys():
        if "-dropout" in key:
            if exp[key] != 0:
                return True
    return False

def has_ln(exp):
    for key in exp.keys():
        if "layer_normalization" in key:
            return exp[key]
    return False

filters_dict = {
    "rd_only" : lambda exp: not has_dropout(exp) and not has_ln(exp),
    "d_only" : lambda exp: not has_recurrent_dropout(exp) and not has_ln(exp),
    "ln_only" : lambda exp: not has_recurrent_dropout(exp) and not has_dropout(exp),
    "no_rd" : lambda exp: not has_recurrent_dropout(exp),
    "no_dropout": lambda exp: not has_dropout(exp),
}

filter = filters_dict[FILTER]

exp_samples = {}
for i, exp in enumerate(grid_search_results):
    exp_name = get_exp_name(exp)
    if not filter(exp):
        continue
    exp_samples[exp_name] = []
    folds = exp["metrics_per_fold"]
    for fold in folds:
        exp_samples[exp_name].extend(fold["root_mean_squared_error"]["unscaled"].metric_per_sample)
    #print(len(exp_samples[exp_name]))

#for exp_name, samples in exp_samples.items():
#    print(f"{exp_name}: {len(samples)}")
#    print(f"mean: {sum(samples)/len(samples)}")
#    print(f"std: {pd.Series(samples).std()}")
    


In [59]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

samples = list(exp_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(10, 450)


FriedmanchisquareResult(statistic=2533.8390303030283, pvalue=0.0)

In [60]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.85247623e-75 1.85247623e-75 1.85247623e-75 1.85247623e-75
 1.85247623e-75 1.85247623e-75 1.85247623e-75 1.85247623e-75
 1.85247623e-75 1.85247623e-75 1.85247623e-75 1.85247623e-75
 1.85247623e-75 1.85247623e-75 1.85247623e-75 1.85247623e-75
 7.37028723e-68 2.83308085e-61 2.83154103e-58 3.25028378e-53
 1.72598171e-49 2.75580890e-46 1.37009471e-41 4.42692322e-40
 6.11755785e-39 6.23505304e-39 4.27317898e-37 1.40379838e-35
 7.75188280e-31 4.37983141e-29 1.95466006e-17 4.22756713e-14
 1.17528619e-11 3.27609715e-11 9.39597757e-11 1.07710602e-10
 1.08410498e-08 1.16402480e-06 7.72757717e-05 6.53102094e-04
 1.88916205e-03 1.22573381e-02 2.97139339e-02 5.00984304e-01
 7.30310247e-01]
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491e-75 is st. significant
1.8524762278349491

(array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True, False, False, False]),
 array([8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        8.33614303e-74, 8.33614303e-74, 8.33614303e-74, 8.33614303e-74,
        2.13738330e-66, 7.93262638e-60, 7.64516078e-57, 8.45073783e-52,
        4.31495427e-48, 6.61394135e-45, 3.15121782e-40, 9.73923109e-39,
        1.28468715e-37, 1.28468715e-37, 8.11904007e-36, 2.52683708e-34,
        1.31782008e-29, 7.00773026e-28, 2.93199009e-16, 5.91859398e-13,
        1.52787205e-10, 3.93131658e-10, 1.03355753e-09, 1.07710602e

In [61]:
diag_df = pd.DataFrame(exp_samples)
diag_df.columns

Index(['Model-layer_normalization rd=0.0 dropout=0.0',
       'Model-layer_normalization rd=0.0 dropout=0.05',
       'Model-layer_normalization rd=0.0 dropout=0.1',
       'Model-layer_normalization rd=0.0 dropout=0.2',
       'Model-layer_normalization rd=0.0 dropout=0.3',
       'Not model-layer_normalization rd=0.0 dropout=0.0',
       'Not model-layer_normalization rd=0.0 dropout=0.05',
       'Not model-layer_normalization rd=0.0 dropout=0.1',
       'Not model-layer_normalization rd=0.0 dropout=0.2',
       'Not model-layer_normalization rd=0.0 dropout=0.3'],
      dtype='object')

In [62]:
exp_samples.values()

dict_values([[0.86452264, 0.8306428, 0.86960936, 0.8294144, 1.0834703, 0.8693929, 0.8378433, 0.8811428, 0.9947093, 0.91291004, 1.002704, 1.0789347, 1.2533572, 1.4135102, 0.88313276, 0.8844523, 0.9197078, 0.88855356, 0.8263152, 0.9152234, 1.288227, 0.87900716, 0.82755333, 0.9354167, 1.4267446, 1.2255495, 1.0886838, 0.922656, 1.1006953, 1.083027, 0.99578387, 0.8323362, 0.87264997, 0.89362854, 1.0656873, 1.1337405, 1.0493717, 0.8773512, 1.1291276, 0.86991304, 1.1788114, 0.85638505, 1.0780281, 0.90322906, 0.8266154, 0.9046537, 0.8240992, 0.7212142, 1.1017846, 0.6547753, 0.97596353, 1.2638031, 0.6915619, 0.7973435, 1.3092722, 1.3671781, 1.2471988, 0.80953157, 0.6672723, 1.050263, 0.8009997, 0.6956952, 0.6486081, 1.3971654, 1.0200113, 1.0957421, 0.66329944, 1.184682, 0.9581723, 1.2603213, 1.1848214, 0.98171586, 1.0375352, 1.1323042, 1.0363272, 0.91658807, 0.8231577, 0.91970813, 0.97901016, 0.77659076, 0.9785806, 0.86066705, 0.8070041, 0.81091255, 1.006898, 0.75154346, 0.6704399, 0.67980206, 

In [63]:
x=np.array(list(exp_samples.values()))
treatment_names = list(exp_samples.keys())
print(treatment_names)
x = x.T
diagram = Diagram(x,
    treatment_names=treatment_names,
    maximize_outcome = False,
)

diagram.to_file(
    "critdd_no_rec_drop.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

def print_sorted_ranks(diagram):
    print("\nAverage ranks:")
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    for name, avg_r in sorted_treatments:
        print(f"{name}: {avg_r}")
    
print_sorted_ranks(diagram)


['Model-layer_normalization rd=0.0 dropout=0.0', 'Model-layer_normalization rd=0.0 dropout=0.05', 'Model-layer_normalization rd=0.0 dropout=0.1', 'Model-layer_normalization rd=0.0 dropout=0.2', 'Model-layer_normalization rd=0.0 dropout=0.3', 'Not model-layer_normalization rd=0.0 dropout=0.0', 'Not model-layer_normalization rd=0.0 dropout=0.05', 'Not model-layer_normalization rd=0.0 dropout=0.1', 'Not model-layer_normalization rd=0.0 dropout=0.2', 'Not model-layer_normalization rd=0.0 dropout=0.3']

Average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 1.38
Model-layer_normalization rd=0.0 dropout=0.0: 1.62
Model-layer_normalization rd=0.0 dropout=0.05: 5.168888888888889
Not model-layer_normalization rd=0.0 dropout=0.1: 5.195555555555556
Not model-layer_normalization rd=0.0 dropout=0.05: 5.208888888888889
Model-layer_normalization rd=0.0 dropout=0.1: 6.182222222222222
Model-layer_normalization rd=0.0 dropout=0.2: 6.566666666666666
Not model-layer_normalization rd=0.0 dropout=

In [29]:
def get_models_without_dropout(diagram: Diagram):
    """Gets the best optimizer for each type (Adam, RMSprop, SGD, ...)"""
    sorted_treatments = sorted(zip(diagram.treatment_names, diagram.average_ranks), key=lambda x: x[1])
    no_dropout_models = {}
    for name, avg_rank in sorted_treatments:
        if name.endswith("dropout=0.0"):
            #name = re.sub(" dropout=0.0", "", name)
            no_dropout_models[name] = (name, avg_rank)
    return no_dropout_models

def get_partial_diagram(subset_names, samples):
    x = np.array([samples[name] for name in subset_names])
    x = x.T
    return Diagram(x,
        treatment_names=subset_names,
        maximize_outcome = False,
    )

no_dropout_models = get_models_without_dropout(diagram)
partial_diagram = get_partial_diagram([name for name, _ in no_dropout_models.values()], exp_samples)

print("Previous average ranks:")
for name, avg_r in no_dropout_models.values():
    print(f"{name}: {avg_r}")

print("\nNew average ranks:")
sorted_treatments = sorted(zip(partial_diagram.treatment_names, partial_diagram.average_ranks), key=lambda x: x[1])
for name, avg_r in sorted_treatments:
    print(f"{name}: {avg_r}")

partial_diagram.to_file(
    "critdd_layernorm_no_drop.tex",
    alpha=0.05,
    adjustment="holm",
    reverse_x=True,
)

Previous average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd=0.2 dropout=0.0: 5.3933333333333335
Model-layer_normalization rd=0.05 dropout=0.0: 5.595555555555555
Model-layer_normalization rd=0.3 dropout=0.0: 6.0311111111111115
Not model-layer_normalization rd=0.3 dropout=0.0: 6.153333333333333
Not model-layer_normalization rd=0.1 dropout=0.0: 6.388888888888889
Not model-layer_normalization rd=0.05 dropout=0.0: 6.753333333333333

New average ranks:
Not model-layer_normalization rd=0.0 dropout=0.0: 4.302222222222222
Model-layer_normalization rd=0.2 dropout=0.0: 4.377777777777778
Model-layer_normalization rd=0.1 dropout=0.0: 4.988888888888889
Model-layer_normalization rd=0.0 dropout=0.0: 5.015555555555555
Not model-layer_normalization rd